In [1]:
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights, synonyms, skill_cat, pandas as pd

In [18]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)
# job_descriptions = skill_weights.get_job_descriptions(raw_job_details)

In [19]:
len(jobs)

100

In [20]:
jobs.head(3)

,company_name,location,role,url,excitement
id,,,,,
bd59469c-29f4-41b9-9001-5b7902d766e1,Ansys,"Madrid, Community of Madrid, Spain",R&D Engineer II - Python f/m,https://www.linkedin.com/jobs/search/?currentJ...,4
0124e218-2d33-41e8-907b-5228ea386455,Treibacher Industrie AG,"Althofen, Carinthia, Austria",AI/ML Specialist – Software Engineer (m/f/d),https://www.linkedin.com/jobs/search/?geoId=91...,4
35b60a7d-f9a3-4f63-b8ab-4db8db49fca8,Helsing,"Paris, Île-de-France, France",Deployed AI Engineer,https://www.linkedin.com/jobs/search/?currentJ...,5


# What is the goal here?
How well do we categorize the skills in listings?

In [53]:
skill_cat._refresh()
synonyms._refresh()

In [54]:
job_skills_data = skill_weights.get_job_skills_data(raw_job_details=raw_job_details, collapse_teal_names=False)
job_skills_data["category"] = [skill_cat.skill_to_categories.get(skill, {"UNK"}) for skill in job_skills_data.index.get_level_values(2)]
jsd_count = job_skills_data.pop("count")
job_skills_data["share of job"] = jsd_count / (job_skills_data["category"].str.len() * jsd_count.groupby(level=0).sum())
job_skills_data = job_skills_data.explode("category").reset_index().set_index(["id", "teal category", "category", "skill", "tealPhrase"]).sort_index()

In [55]:
job_skills_data

share of job
id                                   teal category        category skill                 tealPhrase                    
0124e218-2d33-41e8-907b-5228ea386455 emphasis             Project  managing requirements requirements          0.018519
                                                          UNK      excellent             excellent             0.018519
                                                                   knowledge             knowledge             0.037037
                                     functional abilities Academic computer science      computer science      0.009259
                                                          Admin    automation            automating            0.018519
...                                                                                                                 ...
fdd15faa-0804-4b07-9f55-715436213e75 general abilities    UNK      team performance      team performance      0.007874
                                                                   trusted               trusted               0.007874
                                                          Unix     containerization      deployment            0.007874
                                     platform             Code     python                python                0.003937
                                                          Python   python                python                0.003937

[6677 rows x 1 columns]

In [56]:
global_shares = (job_skills_data.groupby(level=[1,2,3,4]).sum() / job_skills_data.sum())["share of job"].sort_values(ascending=False)

In [57]:
global_shares

teal category         category             skill             tealPhrase             
emphasis              UNK                  experience        experience                 0.063014
functional abilities  ML                   machine learning  machine learning           0.036887
                                                             ai                         0.019050
general abilities     Databases            data              data                       0.018170
                      Data Infrastructure  data              data                       0.018170
                                                                                          ...   
functional abilities  Soft                 agile             agile methodology          0.000050
company attributes    Soft                 agile             agile teams                0.000050
functional abilities  Project              agile             agile methodology          0.000050
                      Unix                

In [58]:
uncategorized = global_shares.loc[:, "UNK"]

In [59]:
# What fraction of mentioned skills are uncategorized?
uncategorized.sum() / global_shares.sum()

0.5397382347899691

In [60]:
# How many times more popular are uncategorized skills vs the average?
uncategorized.mean() / global_shares.mean()

0.6256780211213121

In [70]:
# What fraction of skills in each teal category are uncategorized?
(uncategorized.groupby("teal category").sum() / global_shares.groupby("teal category").sum()).sort_values(ascending=False)

teal category
certifications          1.000000
company attributes      0.959138
emphasis                0.956904
functional abilities    0.501841
general abilities       0.323855
platform                0.231662
Name: share of job, dtype: float64

In [61]:
# What share of all skills are uncategorized, broken down by teal category?
# Read: {'functional abilities': 0.22...} -> 22% of all skill mentions got the teal category 'functional abilities' and have not been categorized by me.
uncategorized.groupby("teal category").sum().sort_values(ascending=False)

teal category
functional abilities    0.227514
emphasis                0.189847
general abilities       0.090150
company attributes      0.020059
platform                0.011094
certifications          0.001074
Name: share of job, dtype: float64

In [72]:
# Among the (currently) most popular teal category, what are the most popular skills?
# 'white space' is an artifact of skills-ml, a tool used by teal.
# Read: {'computer vision':0.0033} -> Computer Vision is a functional ability I have not categorized, and it accounts for 0.33% of all skill mentions.
uncategorized.loc["functional abilities"].sort_values(ascending=False).head(20)

skill                     tealPhrase              
white space               white space                 0.005680
paid                      paid                        0.003701
computer vision           computer vision             0.003338
monitoring                monitoring                  0.002953
enterprise                enterprise                  0.002647
events                    events                      0.002444
consideration             consideration               0.002109
professional development  professional development    0.001855
mental health             mental health               0.001730
acquisition               acquisition                 0.001689
foundry                   foundry                     0.001662
development experience    development experience      0.001498
signal processing         signal processing           0.001477
real time                 real time                   0.001454
interview process         interview process           0.001414
hiri